Creating your First Graph and Running it in a Session

In [2]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

f = x*x*y + y + 2

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


Global variable initializer

In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


Managing Graph

In [13]:
# Any node that is created is automatically added to the default graph
x1 = tf.Variable(1, name="x1")
x1.graph is tf.get_default_graph()

True

In [16]:
# Managing multiple independent graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2, name="x2")
    
x2.graph is graph

True

In [17]:
x2.graph is tf.get_default_graph()

False

Lifecycle of a Node Value

All node values are dropped between graph runs, except variable values which are maintained by the session across graph runs.

In [22]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

# Below code evaluates w & x twice

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


If one wants to evaluate y & Z effectively, without evaluating w & x twice, one must evaluate y & z in just one graph run

In [23]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


Linear Regression with TensorFlow

In [24]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape

downloading Cal. housing from http://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.tgz to /home/dimension/scikit_learn_data


In [26]:
print(m, n)

20640 8


In [63]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

XT = tf.transpose(X)

# theta = ((XT * X) ^ -1) * XT * y

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
    print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


Gradient Descent

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaler.transform(housing.data)]
n_epochs = 1500
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name="mse")

In [72]:
# Manually computing the gradient
# gradients = 2/m * tf.matmul(tf.transpose(X), error)
# training_op = tf.assign(theta, theta - learning_rate * gradients)

# Using autodiff
# gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)

# Using an optimizer 
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# training_op = optimizer.minimize(mse)

# Momentum optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
    print(best_theta)

Epoch 0 MSE =  12.47409
Epoch 100 MSE =  0.5292289
Epoch 200 MSE =  0.52446055
Epoch 300 MSE =  0.5243309
Epoch 400 MSE =  0.52432215
Epoch 500 MSE =  0.524321
Epoch 600 MSE =  0.52432096
Epoch 700 MSE =  0.524321
Epoch 800 MSE =  0.5243208
Epoch 900 MSE =  0.52432096
Epoch 1000 MSE =  0.52432084
Epoch 1100 MSE =  0.5243209
Epoch 1200 MSE =  0.5243209
Epoch 1300 MSE =  0.5243209
Epoch 1400 MSE =  0.5243209
[[ 2.068558  ]
 [ 0.82962054]
 [ 0.11875187]
 [-0.265529  ]
 [ 0.30569798]
 [-0.00450293]
 [-0.03932633]
 [-0.89988273]
 [-0.87053823]]
